In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv('CHATGPT_API_KEY'))

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[]


In [6]:

instructions = """
Identify if a pedestrian is present within a red bounding box in an image.

Criteria:

* Presence: Any part of a real human within the box constitutes a "pedestrian."
* Context: The pedestrian may be on foot, riding a vehicle (e.g., bicycle, motorcycle), or inside a vehicle.
* Exclusion: Pedestrians depicted in signs, posters, or other media are not considered "real" pedestrians.

Output:

* "ped": If a pedestrian is present within the bounding box.
* "fa": If no pedestrian is present within the bounding box.

Note: The assistant should focus on the specific area defined by the red bounding box and ignore elements outside of it.
"""

my_assistant = client.beta.assistants.create(
    instructions=instructions,
    name="FaRemoval",
    model="gpt-4o",
)

In [7]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_GSfa8WShZkY8LIzCvKmUN3mZ', created_at=1725183519, description=None, instructions='\nIdentify if a pedestrian is present within a red bounding box in an image.\n\nCriteria:\n\n* Presence: Any part of a real human within the box constitutes a "pedestrian."\n* Context: The pedestrian may be on foot, riding a vehicle (e.g., bicycle, motorcycle), or inside a vehicle.\n* Exclusion: Pedestrians depicted in signs, posters, or other media are not considered "real" pedestrians.\n\nOutput:\n\n* "ped": If a pedestrian is present within the bounding box.\n* "fa": If no pedestrian is present within the bounding box.\n\nNote: The assistant should focus on the specific area defined by the red bounding box and ignore elements outside of it.\n', metadata={}, model='gpt-4o', name='FaRemoval', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0), Assistant(id='asst_GaHNh5kxLuLW6W4Q9FCk6v

In [23]:
assistant = client.beta.assistants.retrieve("asst_GSfa8WShZkY8LIzCvKmUN3mZ")

In [24]:
print(assistant.model)

gpt-4o


In [25]:
file1 = client.files.create(
  file=open("/Users/tomercohen/Downloads/crop2vec_chatgpt/Manual_Tagged/crops_with_bb_false_ManualFalse/1684255282417818_det_3491.png", "rb"),
  purpose="vision"
)

file2 = client.files.create(
  file=open("/Users/tomercohen/Downloads/crop2vec_chatgpt/Manual_Tagged/crops_with_bb_false_ManualFalse/1684255193262310_det_5659.png", "rb"),
  purpose="vision"
)

thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What is the difference between these images?"
        },
        {
          "type": "image_file",
          "image_file": {"file_id": file1.id, "detail": "high"}
        },
        {
          "type": "image_file",
          "image_file": {"file_id": file2.id}
        },
      ],
    }
  ]
)

In [26]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [29]:
run.id

'run_sYpuON8ARGS3vxg0ClcCbUzZ'

In [31]:
run_result = client.beta.threads.runs.retrieve(run_id='run_sYpuON8ARGS3vxg0ClcCbUzZ', thread_id=thread.id)

In [33]:
run_result['messages']

TypeError: 'Run' object is not subscriptable